# ==========================================
# Jewellery Recommendation System (Tkinter)
# ==========================================

In [1]:


import os
import re
import webbrowser
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ==========================================
# Dataset Path
# ==========================================

In [2]:
DATA_DIR = "C:\\jewellery"
CSV_FILE = os.path.join(DATA_DIR, "metadata.csv")

if not os.path.exists(CSV_FILE):
    raise FileNotFoundError("metadata.csv not found")

# ==========================================
# Load Dataset
# ==========================================

In [3]:
df = pd.read_csv(CSV_FILE)

df.fillna("", inplace=True)
df.drop_duplicates(subset="Article Id", inplace=True)
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df = df[df["Price"].notna() & (df["Price"] > 0)]

text_columns = [
    "Product Name", "Description", "Features",
    "Product Type", "Brand", "Country of Origin"
]

for col in text_columns:
    df[col] = df[col].str.lower().str.strip()

df["combined_features"] = (
    df["Product Name"] + " " +
    df["Description"] + " " +
    df["Features"] + " " +
    df["Product Type"] + " " +
    df["Brand"] + " " +
    df["Country of Origin"]
)

# ==========================================
# Vectorization
# ==========================================

In [4]:
vectorizer = TfidfVectorizer(stop_words="english")
feature_vectors = vectorizer.fit_transform(df["combined_features"])

In [5]:
print("Feature Vectors Generated")
print("Number of products:", feature_vectors.shape[0])
print("Number of features (vocabulary size):", feature_vectors.shape[1])

Feature Vectors Generated
Number of products: 1926
Number of features (vocabulary size): 5214


# ==========================================
# Helper Functions
# ==========================================

In [6]:
def open_product(url):
    if url:
        webbrowser.open(url)

def highlight_keywords(text, keywords):
    for word in keywords:
        if word in text:
            text = text.replace(word, f"[{word.upper()}]")
    return text


# ==========================================
# Recommendation Logic
# ==========================================

In [7]:
def recommend_jewellery(user_query, sort_choice, top_n=5):

    user_query = user_query.lower()

    # -------- PRICE EXTRACTION --------
    max_budget = None
    min_budget = None

    lower_match = re.search(r'(under|below|upto)\s*(\d+)', user_query)
    upper_match = re.search(r'(above|over|higher than|more than)\s*(\d+)', user_query)

    if lower_match:
        max_budget = int(lower_match.group(2))

    if upper_match:
        min_budget = int(upper_match.group(2))

    # -------- CLEAN QUERY --------
    clean_query = re.sub(
        r'(under|below|upto|above|over|higher than|more than)\s*\d+',
        '',
        user_query
    ).strip()

    # -------- VECTORIZE QUERY --------
    user_vector = vectorizer.transform([clean_query])

    similarities = cosine_similarity(user_vector, feature_vectors)[0]

    # Work on a COPY (important)
    temp_df = df.copy()
    temp_df["similarity"] = similarities

    # -------- SIMILARITY FILTER --------
    temp_df = temp_df[temp_df["similarity"] > 0.05]

    # -------- PRICE FILTER --------
    if max_budget is not None:
        temp_df = temp_df[temp_df["Price"] <= max_budget]

    if min_budget is not None:
        temp_df = temp_df[temp_df["Price"] >= min_budget]

    # -------- NO RELEVANT PRODUCTS FOUND --------
    if temp_df.empty:
        print("No relevant products found for this query.")
        return pd.DataFrame(), clean_query.split()


    # -------- SORT BY SIMILARITY FIRST --------
    temp_df = temp_df.sort_values("similarity", ascending=False)

    # Take top relevant candidates
    top_relevant = temp_df.head(20)

    # -------- USER SORT OPTION --------
    if sort_choice == "Price: Low to High":
        top_relevant = top_relevant.sort_values("Price", ascending=True)
    elif sort_choice == "Price: High to Low":
        top_relevant = top_relevant.sort_values("Price", ascending=False)
    # else: Similarity (already sorted)

    # -------- CLEAN DEBUG PRINT (--------
    print("\n--- Recommendation Debug ---")
    print("Query:", clean_query)
    print("Min Budget:", min_budget, "Max Budget:", max_budget)
    print(top_relevant[["Product Name", "Price", "similarity"]].head(top_n))

    return top_relevant.head(top_n), clean_query.split()


In [8]:
root = tk.Tk()
root.title("Jewellery Recommendation System")
root.geometry("1000x650")
root.configure(bg="#f4f4f4")

image_refs = []

# ---------- HEADER ----------

In [9]:
tk.Label(
    root,
    text="Jewellery Recommendation System",
    font=("Arial", 20, "bold"),
    bg="#f4f4f4"
).pack(pady=15)

# ---------- CONTROL PANEL ----------

In [10]:
control_frame = tk.Frame(root, bg="#f4f4f4")
control_frame.pack(pady=5)

entry = tk.Entry(control_frame, width=45, font=("Arial", 12))
entry.grid(row=0, column=0, padx=10)

sort_option = tk.StringVar(value="Similarity")

tk.OptionMenu(
    control_frame,
    sort_option,
    "Similarity",
    "Price: Low to High",
    "Price: High to Low"
).grid(row=0, column=1, padx=10)

tk.Button(
    control_frame,
    text="Recommend",
    font=("Arial", 11, "bold"),
    command=lambda: on_recommend()
).grid(row=0, column=2, padx=10)

# ---------- SCROLLABLE RESULTS ----------

In [11]:
canvas = tk.Canvas(root, bg="#f4f4f4", highlightthickness=0)
scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)

results_frame = tk.Frame(canvas, bg="#f4f4f4")

results_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

canvas.create_window((0, 0), window=results_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.pack(side="left", fill="both", expand=True, padx=10)
scrollbar.pack(side="right", fill="y")

# ==========================================
# Button Action
# ==========================================

In [ ]:
def on_recommend():
    global image_refs
    image_refs.clear()

    for widget in results_frame.winfo_children():
        widget.destroy()

    query = entry.get().strip()
    if not query:
        messagebox.showwarning("Input Error", "Please enter a jewellery preference")
        return

    # ---- GET RECOMMENDATIONS FIRST ----
    results, keywords = recommend_jewellery(query, sort_option.get())

    # ---- HANDLE NO RESULTS CASE ----
    if results.empty:
        messagebox.showinfo("No Results", "No matching jewellery found for your query.")
        return

    # ---- DISPLAY RESULTS ----
    for _, row in results.iterrows():
        card = tk.Frame(
            results_frame,
            bg="white",
            bd=1,
            relief="solid",
            padx=10,
            pady=10
        )
        card.pack(fill="x", pady=6)

        # Image
        img_path = os.path.join(DATA_DIR, row["file_name"])
        if os.path.exists(img_path):
            img = Image.open(img_path).resize((120, 120))
            img = ImageTk.PhotoImage(img)
            image_refs.append(img)

            img_label = tk.Label(card, image=img, cursor="hand2", bg="white")
            img_label.pack(side="left", padx=10)
            img_label.bind(
                "<Button-1>",
                lambda e, url=row["Product Url"]: open_product(url)
            )

        # Info
        info = tk.Frame(card, bg="white")
        info.pack(side="left", fill="x")

        name = highlight_keywords(row["Product Name"], keywords)

        tk.Label(info, text=name.title(), font=("Arial", 14, "bold"), bg="white").pack(anchor="w")
        tk.Label(info, text=f"Brand: {row['Brand'].title()}", bg="white").pack(anchor="w")
        tk.Label(info, text=f"Type: {row['Product Type'].title()}", bg="white").pack(anchor="w")
        tk.Label(info, text=f"Price: $ {int(row['Price'])}", fg="green", bg="white", font=("Arial", 11, "bold")).pack(anchor="w")
        tk.Label(info, text=f"Similarity: {row['similarity']:.2f}", fg="gray", bg="white").pack(anchor="w")

root.mainloop()



--- Recommendation Debug ---
Query: blue stone pendent
Min Budget: None Max Budget: 400
                                           Product Name  Price  similarity
6340              chopra gems stone pendant locket blue    210    0.291287
4368       chopra gems stone blue sapphire pendant blue    308    0.279681
3592  chopra gems stone blue sapphire pendant blue (...    356    0.279488
3193   chopra gems stone original blue sapphire pendant    395    0.263098
6745  jn handicraft alloy copper-plated blue jewel s...    184    0.154904

--- Recommendation Debug ---
Query: blue stone pendent
Min Budget: None Max Budget: 400
                                           Product Name  Price  similarity
6798  the stone aisle premium rakhi combo set of 3 f...    179    0.108035
6742  jn handicraft alloy copper-plated blue jewel s...    184    0.154904
6745  jn handicraft alloy copper-plated blue jewel s...    184    0.154904
6340              chopra gems stone pendant locket blue    210    0.2912